## Company: Walmart
#### Group Members: Hala Sedki, Reem Abdelghany, & Ebraam Hani
#### Financial Data Analysis (Sales)

## Exploring the Data

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import pipeline
from sklearn.preprocessing import LabelEncoder
from scipy import stats 
import time

In [2]:
walmart_df = pd.read_csv('walmart_data.csv')

In [3]:
walmart_df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


## Data Analysis

In [4]:
walmart_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9800 non-null   int64  
 1   Order ID       9800 non-null   object 
 2   Order Date     9800 non-null   object 
 3   Ship Date      9800 non-null   object 
 4   Ship Mode      9800 non-null   object 
 5   Customer ID    9800 non-null   object 
 6   Customer Name  9800 non-null   object 
 7   Segment        9800 non-null   object 
 8   Country        9800 non-null   object 
 9   City           9800 non-null   object 
 10  State          9800 non-null   object 
 11  Postal Code    9789 non-null   float64
 12  Region         9800 non-null   object 
 13  Product ID     9800 non-null   object 
 14  Category       9800 non-null   object 
 15  Sub-Category   9800 non-null   object 
 16  Product Name   9800 non-null   object 
 17  Sales          9800 non-null   float64
dtypes: float

### Check for Missing Values

In [8]:
walmart_df.loc[walmart_df["Postal Code"].isna(),['Country','City','State','Postal Code']]

,Country,City,State,Postal Code
2234,United States,Burlington,Vermont,NaN
5274,United States,Burlington,Vermont,NaN
8798,United States,Burlington,Vermont,NaN
9146,United States,Burlington,Vermont,NaN
9147,United States,Burlington,Vermont,NaN
9148,United States,Burlington,Vermont,NaN
9386,United States,Burlington,Vermont,NaN
9387,United States,Burlington,Vermont,NaN
9388,United States,Burlington,Vermont,NaN
9389,United States,Burlington,Vermont,NaN


In [9]:
walmart_df.loc[(walmart_df['City']=='Burlington') & (walmart_df['State']=='Vermont'),['Country','Postal Code']]

,Country,Postal Code
2234,United States,NaN
5274,United States,NaN
8798,United States,NaN
9146,United States,NaN
9147,United States,NaN
9148,United States,NaN
9386,United States,NaN
9387,United States,NaN
9388,United States,NaN
9389,United States,NaN


In [12]:
train = walmart_df.copy()
train['Postal Code'] = train['Postal Code'].fillna(5401) # leading zeros in decimal integer literals are not permitted so we use 5401

I checked the code for the city of Burlington as it was the one with missing values and found the postal code as 05401, this is why in the fillna, I assigned the postal code as 5401

In [13]:
train.isna().sum().sum()

0

### Drop Unwanted Features

In [14]:
train.drop(['Row ID','Customer Name','Order ID','Product ID'],axis=1,inplace=True)

### Check for Duplicate Data

In [15]:
train[train.duplicated()]

,Order Date,Ship Date,Ship Mode,Customer ID,Segment,Country,City,State,Postal Code,Region,Category,Sub-Category,Product Name,Sales
3406,23/04/2015,27/04/2015,Standard Class,LB-16795,Home Office,United States,Columbus,Ohio,43229.0,East,Furniture,Chairs,Global Leather Highback Executive Chair with P...,281.372


In [16]:
train.drop_duplicates(inplace=True)
train.duplicated().sum()

0

### Convert Data Types

In [17]:
train['Order Date'] = pd.to_datetime(train['Order Date'], format='%d/%m/%Y') 
train['Ship Date'] = pd.to_datetime(train['Ship Date'], format='%d/%m/%Y')  
train['Postal Code'] = train['Postal Code'].astype(int)

The above was done to format the date correctly in order to later visualize it correctly, so we assigned it the datetime data type

In [18]:
train.insert(loc=4,  column='order_month_year',value=train['Order Date'].dt.to_period('M'))
train.insert(loc=5, column='ship_month_year', value=train['Ship Date'].dt.to_period('M'))

train.insert(loc=6, column='order_day', value=train['Order Date'].dt.day)
train.insert(loc=7, column='order_month', value=train['Order Date'].dt.month)
train.insert(loc=8, column='order_year', value=train['Order Date'].dt.year)

train.insert(loc=9, column='ship_day', value=train['Ship Date'].dt.day)
train.insert(loc=10, column='ship_month', value=train['Ship Date'].dt.month)
train.insert(loc=11, column='ship_year', value=train['Ship Date'].dt.year)

Here the same was done but we seperated the month & year

In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9799 entries, 0 to 9799
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Order Date        9799 non-null   datetime64[ns]
 1   Ship Date         9799 non-null   datetime64[ns]
 2   Ship Mode         9799 non-null   object        
 3   Customer ID       9799 non-null   object        
 4   order_month_year  9799 non-null   period[M]     
 5   ship_month_year   9799 non-null   period[M]     
 6   order_day         9799 non-null   int32         
 7   order_month       9799 non-null   int32         
 8   order_year        9799 non-null   int32         
 9   ship_day          9799 non-null   int32         
 10  ship_month        9799 non-null   int32         
 11  ship_year         9799 non-null   int32         
 12  Segment           9799 non-null   object        
 13  Country           9799 non-null   object        
 14  City              9799 non-nu

In [20]:
pd.set_option('display.max_columns', None)
train.head(5)

,Order Date,Ship Date,Ship Mode,Customer ID,order_month_year,ship_month_year,order_day,order_month,order_year,ship_day,ship_month,ship_year,Segment,Country,City,State,Postal Code,Region,Category,Sub-Category,Product Name,Sales
0,2017-11-08,2017-11-11,Second Class,CG-12520,2017-11,2017-11,8,11,2017,11,11,2017,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2017-11-08,2017-11-11,Second Class,CG-12520,2017-11,2017-11,8,11,2017,11,11,2017,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,2017-06-12,2017-06-16,Second Class,DV-13045,2017-06,2017-06,12,6,2017,16,6,2017,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,2016-10-11,2016-10-18,Standard Class,SO-20335,2016-10,2016-10,11,10,2016,18,10,2016,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,2016-10-11,2016-10-18,Standard Class,SO-20335,2016-10,2016-10,11,10,2016,18,10,2016,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680
